## 제주도사랑하는모임 전기차 키워드로 크롤링하기

#### 환경설치

In [2]:
# !pip install pyperclip
# !pip install selenium

  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=1c7764ae02bf4adfd7ea82c0b1947d079ab30d33f0c8442d6f7c747d87674770
  Stored in directory: c:\users\alsdn\appdata\local\pip\cache\wheels\0c\09\9e\49e21a6840ef7955b06d47394afef0058f0378c0914e48b8b8
Successfully built pyperclip


### User libaray

In [1]:
from selenium import webdriver
import time
import pandas as pd
from urllib.parse import quote
from selenium.webdriver.common.keys import Keys
import time
import pyperclip

### driver open


In [ ]:
driver = webdriver.Chrome('chromedriver.exe')  # 윈도우
time.sleep(1)

In [3]:
driver = webdriver.Chrome('/media/minwooking/9C33-6BBD/WorkSpace/Jeju_Electric_Vehicle_analysis/chromedriver')  # 리눅스
time.sleep(1)

/tmp/ipykernel_106718/3732133635.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/media/minwooking/9C33-6BBD/WorkSpace/Jeju_Electric_Vehicle_analysis/chromedriver')  # 리눅스


### driver close

In [8]:
driver.quit()

### 아이디 비밀번호 불러오기

In [ ]:
with open('F:\\WorkSpace\\mykey\\Naver_pw.txt') as file: # 윈도우
    PASSWORD = file.read()
with open('F:\\WorkSpace\\mykey\\Naver_ID.txt') as file: # 윈도우 
    ID = file.read()

In [5]:
## 리눅스
with open('/media/minwooking/9C33-6BBD/WorkSpace/mykey/Naver_ID.txt') as file: 
    password = file.read()
with open('/media/minwooking/9C33-6BBD/WorkSpace/mykey/Naver_pw.txt') as file:
    ID = file.read()

### 로그인

In [4]:
url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com' 
uid = ID
upw = password
#네이버 로그인 페이지로 이동 
driver.get(url) 
time.sleep(2) #로딩 대기 

#아이디 입력폼 
tag_id = driver.find_element_by_name('id') 
#패스워드 입력폼 
tag_pw = driver.find_element_by_name('pw')

# id 입력 
# 입력폼 클릭 -> paperclip에 선언한 uid 내용 복사 -> 붙여넣기 
tag_id.click() 
pyperclip.copy(uid) 
tag_id.send_keys(Keys.CONTROL, 'v') 
time.sleep(1) 

# pw 입력 # 입력폼 클릭 -> paperclip에 선언한 upw 내용 복사 -> 붙여넣기
tag_pw.click() 
pyperclip.copy(upw) 
tag_pw.send_keys(Keys.CONTROL, 'v')
time.sleep(1) 

#로그인 버튼 클릭 
login_btn = driver.find_element_by_id('log.login') 
login_btn.click() 
time.sleep(2)



NameError: name 'ID' is not defined

### 전기차 검색 코드

In [5]:
driver.get('https://cafe.naver.com/idiolle')
tag_search = driver.find_element_by_name('query').send_keys('전기차')
driver.find_element_by_name("query").send_keys(Keys.ENTER)
time.sleep(2)
driver.switch_to.frame("cafe_main")

/tmp/ipykernel_106718/3677752914.py:2: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  tag_search = driver.find_element_by_name('query').send_keys('전기차')
/tmp/ipykernel_106718/3677752914.py:3: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  driver.find_element_by_name("query").send_keys(Keys.ENTER)


In [9]:
driver.get('https://cafe.naver.com/ArticleSearchList.nhn?search.clubid=20692649&search.media=0&search.searchdate=all&userDisplay=15&search.option=0&search.sortBy=date&search.searchBy=0&search.query=%C0%FC%B1%E2%C2%F7&search.viewtype=title&search.page=1')
time.sleep(2)
driver.switch_to.frame('cafe_main')
driver.find_elements_by_xpath(f'/html/body/div[1]/div/div[5]/table/tbody/tr[1]/td[1]/div[2]/div/a[1]')[0].get_attribute("href")

C:\Users\alsdn\AppData\Local\Temp/ipykernel_15600/2477498050.py:4: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  driver.find_elements_by_xpath(f'/html/body/div[1]/div/div[5]/table/tbody/tr[1]/td[1]/div[2]/div/a[1]')[0].get_attribute("href")


'https://cafe.naver.com/ca-fe/ArticleRead.nhn?clubid=20692649&page=1&inCafeSearch=true&searchBy=0&query=%EC%A0%84%EA%B8%B0%EC%B0%A8&includeAll=&exclude=&include=&exact=&searchdate=all&media=0&sortBy=date&articleid=1424190&referrerAllArticles=true'

In [13]:
### test
driver.get('https://cafe.naver.com/ca-fe/ArticleRead.nhn?clubid=20692649&page=1&inCafeSearch=true&searchBy=0&query=%EC%A0%84%EA%B8%B0%EC%B0%A8&includeAll=&exclude=&include=&exact=&searchdate=all&media=0&sortBy=date&articleid=1424190&referrerAllArticles=true')
time.sleep(2)
driver.switch_to.frame('cafe_main')
driver.find_element_by_class_name('date').text
driver.find_element_by_class_name('count').text

C:\Users\alsdn\AppData\Local\Temp/ipykernel_15600/1774579828.py:5: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  driver.find_element_by_class_name('date').text


'2022.03.14. 14:38'

### URL 주소 불러오기

In [6]:
address=[]
pops=[]
page= 0
#while page < 0:
while page < 50:
    try:
        page+=1
        driver.get(f"https://cafe.naver.com/idiolle?iframe_url=/ArticleSearchList.nhn%3Fsearch.clubid=10456178%26search.media=0%26search.searchdate=all%26search.exact=%26search.include=%26userDisplay=50%26search.exclude=%26search.option=0%26search.sortBy=date%26search.searchBy=0%26search.includeAll=%26search.query=%C0%FC%B1%E2%C2%F7%26search.viewtype=title%26search.page={page}")
        driver.switch_to.frame("cafe_main")
        try:
            if driver.find_element_by_class_name("nodata").text == '등록된 게시글이 없습니다.':
            # 무한으로 다음페이지로 넘어가는데 만약에 등록된게시글이 없습니다가 나오면 다음페이지로 넘어가는것을 멈춤
                break
        except:
            pass

        b = driver.find_element_by_xpath('/html/body/div[1]/div/div[5]/table/tbody')
        for i in range(0,50):
            address.append(b.find_elements_by_class_name('article')[i].get_attribute('href'))
            pops.append(driver.find_elements_by_class_name('td_view')[i].text)
       
    except:
        pass

/tmp/ipykernel_106718/1133438348.py:12: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  if driver.find_element_by_class_name("nodata").text == '등록된 게시글이 없습니다.':
/tmp/ipykernel_106718/1133438348.py:18: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  b = driver.find_element_by_xpath('/html/body/div[1]/div/div[5]/table/tbody')
/home/minwooking/anaconda3/lib/python3.9/site-packages/selenium/webdriver/remote/webelement.py:464: UserWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  warnings.warn("find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead")
/tmp/ipykernel_106718/1133438348.py:21: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  pops.append(dri

In [12]:
len(dates)

1013

### 게시글 내용,제목,댓글 불러오기

In [15]:
driver.get(address[1])
time.sleep(2)
driver.switch_to.frame('cafe_main')
try :
    if driver.find_element_by_class_name("title_text")==None:
        title = '제목이없습니다.'
except:
    pass
title =driver.find_element_by_class_name("title_text").text
contents =driver.find_elements_by_tag_name('p')
content = ''
for i,item in enumerate(contents):
    content += contents[i].text+'\n'
print(title)
print(content)

C:\Users\alsdn\AppData\Local\Temp/ipykernel_12264/4285009667.py:5: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  if driver.find_element_by_class_name("title_text")==None:
C:\Users\alsdn\AppData\Local\Temp/ipykernel_12264/4285009667.py:9: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  title =driver.find_element_by_class_name("title_text").text
C:\Users\alsdn\AppData\Local\Temp/ipykernel_12264/4285009667.py:10: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  contents =driver.find_elements_by_tag_name('p')


우도전기차에서 직원 모십니다


📢 '최저임금'은 임금의 최저수준을 정하고, 사용자에게 이 수준 이상의 임금을 지급하도록 하는 제도입니다.
- 2022년 최저임금은 시간당 9,160원입니다.
* 업체명 :  우도 전기차

* 연락처 :  010-8663-8393

* 근무지역 :  우도 

* 채용정보 
- 우도에서 전기차 직원모셔요
급여 220만원
점심 식대비 하루 1만원씩 지급합니다
기숙사는 제공하며 공과금이나 따로 내는건 없습니다
아침식사 저녁식사 제공해드립니다
남자 25세에서 45세 사이 
여자 25세에서 35세 사이 모십니다
점심 식사는 근처 식당이나
편의점에서 사서 드셔도 되고요
가게에서 간식을주면 그것만 드셔도됩니다
자세한 사항은 전화문의 부탁드립니다



In [76]:
driver.get(address[0])
time.sleep(2)
driver.switch_to.frame('cafe_main')
comments=driver.find_elements_by_class_name('text_comment')
comment = ''
for k,com in enumerate(comments):
    comment += comments[k].text+'\n'
        
comment  == ''   

/tmp/ipykernel_106718/3517288520.py:4: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  comments=driver.find_elements_by_class_name('text_comment')


True

### 주소 들어가서 내용 크롤링하기

In [108]:
titles = []
contentss = []
link_boards = []
commentss = []
dates=[]
for element in address[:10]:
    try:
        driver.get(element)
        time.sleep(2)
        driver.switch_to.frame('cafe_main')
        try: 
            if driver.find_element_by_class_name("nodata").text == '삭제되었거나 없는 게시글입니다.':
                    break
        except:
                pass
            
        ### 제목 
        try :
            if driver.find_element_by_class_name("title_text").text == None:
                title = '제목이없습니다.'
                pass
            else:
                title =driver.find_element_by_class_name("title_text").text
        except:
            pass
        
        
        ### 내용
        if driver.find_elements_by_tag_name('p')[0].text == None:
            contents =driver.find_element_by_xpath('/html/body/div/div/div/div[2]/div[2]/div[1]/div/div[1]/div/div').find_elements_by_tag_name('b')
        else:
            contents =driver.find_element_by_xpath('/html/body/div/div/div/div[2]/div[2]/div[1]/div/div[1]/div/div').find_elements_by_tag_name('p')
        content = ''
        for j,item in enumerate(contents):
            content += contents[j].text+'\n'
        
        
        ### 댓글
        comments=driver.find_elements_by_class_name('text_comment')
        comment = ''
        for k,com in enumerate(comments):
            comment += comments[k].text+'\n'
        
        if comment  == '':
            comment = '댓글이없습니다.'
        ### 추가
        titles.append(title) ## 제목
        contentss.append(content) ## 내용     
        commentss.append(comment) ## 댓글 어펜드
        link_boards.append(driver.find_element_by_class_name('link_board').text)### 필터링
        dates.append(driver.find_element_by_class_name('date').text)

    except:
        pass

/tmp/ipykernel_106718/2625260118.py:12: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  if driver.find_element_by_class_name("nodata").text == '삭제되었거나 없는 게시글입니다.':
/tmp/ipykernel_106718/2625260118.py:19: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  if driver.find_element_by_class_name("title_text").text == None:
/tmp/ipykernel_106718/2625260118.py:23: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  title =driver.find_element_by_class_name("title_text").text
/tmp/ipykernel_106718/2625260118.py:29: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  if driver.find_elements_by_tag_name('p')[0].text == None:
/tmp/ipykernel_106718/2625260118.py:32: DeprecationWarning: find_element_by_xpa

## 만들어진 결과 데이터프레임으로 만들기

In [109]:
df=pd.DataFrame({"제목":titles ,"내용":contentss,"주소":address,'날짜':dates,'조회수':pops,'게시글':link_boards
                 , "댓글":commentss})
# df.to_csv('제사모전기차크롤링.csv',index=False)
df.head(3)


,제목,내용,주소,날짜,조회수,게시글,댓글
0,요번에 보니..,안에\n\n렌터카 무진장 많이 다니던데..\n\n임산부 노약자 상관없이..\n\n넘...,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,2022.03.15. 20:42,28,실시간 이야기,댓글이없습니다.
1,제주시 애월읍 해안도로 도보 1분 거리 애월 르시엘 아파트 할인 분양(회사 직영),◈ 매물제목 : 애월 르시엘 분양\n◈ 거래유형 : 분양\n◈ 소재지역 : 애...,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,2022.03.15. 15:20,64,■ 분양정보/신축매물,댓글이없습니다.
2,여행 고수님들 추천좀 해주세요,4월22일~25일 딸3명과 엄마까지 총 4인 제주여행을 계획중입니다\n비행기 예약은...,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,2022.03.14. 14:38,42,제주여행 수다&질문,렌트카는 가스 예요.\n평소 본인이 운전하기 편한차로 하세요.\n74시간이면 2박3...


In [92]:
len(link_boards)

10

In [102]:
len(commentss)

10

In [101]:
len(titles)

10

In [100]:
len(contentss)

10

In [96]:
len(address[:10])

10

In [110]:
len(dates[:10])

10

In [99]:
len(pops[:10])

10

In [28]:
# df1 = pd.DataFrame({'제목':titles})
# df2 = pd.DataFrame({'내용':contentss})
# df3 = pd.DataFrame({'주소':address})
# df4 = pd.DataFrame({'시간':dates})
# df5 = pd.DataFrame({'조회수':pops})


In [29]:
# df1.to_csv('제사모1.csv',index=False)
# df2.to_csv('제사모2.csv',index=False)
# df3.to_csv('제사모3.csv',index=False)
# df4.to_csv('제사모4.csv',index=False)
# df5.to_csv('제사모5.csv',index=False)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제목      1001 non-null   object
 1   내용      1001 non-null   object
 2   주소      1001 non-null   object
dtypes: object(3)
memory usage: 23.6+ KB
